In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re

2024-04-30 15:40:02.995758: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## CNN w/ word2vec

In [4]:
df = pd.read_csv('../data/data.csv')
df.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [5]:
def preprocess(text):
    text = re.sub(r'http\S+', '', text)  # Removes URLs that start with http
    text = re.sub(r'www\S+', '', text)   # Removes URLs that start with www

    text = text.lower()  # Lowercase
    tokens = word_tokenize(text)  # Tokenize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stopwords.words('english')]
    return tokens

# Apply preprocessing
df['processed_posts'] = df['posts'].apply(preprocess)

In [6]:
df

,type,posts,processed_posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,"[intj, moment, sportscenter, top, ten, play, e..."
1,ENTP,'I'm finding the lack of me in these posts ver...,"[finding, lack, post, boring, position, often,..."
2,INTP,'Good one _____ https://www.youtube.com/wat...,"[one, course, say, know, blessing, absolutely,..."
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...","[intp, enjoyed, conversation, day, esoteric, g..."
4,ENTJ,'You're fired.|||That's another silly misconce...,"[another, silly, misconception, approaching, l..."
...,...,...,...
8670,ISFP,'https://www.youtube.com/watch?v=t8edHB_h908||...,"[always, think, cat, fi, doms, reason, website..."
8671,ENFP,'So...if this thread already exists someplace ...,"[thread, already, exists, someplace, else, hec..."
8672,INTP,'So many questions when i do these things. I ...,"[many, question, thing, would, take, purple, p..."
8673,INFP,'I am very conflicted right now when it comes ...,"[conflicted, right, come, wanting, child, hone..."


In [9]:
"""
Here we create 4 new columns each containing information about one of the key dichotomies of MBTI
"""

def label_mbti_ie(row):
    if "I" in row['type']:
        return 0
    else:
        return 1
    
def label_mbti_ns(row):
    if "N" in row['type']:
        return 0
    else:
        return 1

def label_mbti_ft(row):
    if "F" in row['type']:
        return 0
    else:
        return 1

def label_mbti_pj(row):
    if "P" in row['type']:
        return 0
    else:
        return 1

df['I/E'] = df.apply(label_mbti_ie, axis=1)
df['N/S'] = df.apply(label_mbti_ns, axis=1)
df['F/T'] = df.apply(label_mbti_ft, axis=1)
df['P/J'] = df.apply(label_mbti_pj, axis=1)

# df['I-E'] = df['Type'].apply(lambda x: 1 if 'E' in x['type'] else 0)
# df['N-S'] = df['Type'].apply(lambda x: 1 if 'S' in x['type'] else 0)
# df['F-T'] = df['Type'].apply(lambda x: 1 if 'T' in x['type'] else 0)
# df['P-J'] = df['Type'].apply(lambda x: 1 if 'J' in x['type'] else 0)

df

,type,posts,processed_posts,I/E,N/S,F/T,P/J
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,"[intj, moment, sportscenter, top, ten, play, e...",0,0,0,1
1,ENTP,'I'm finding the lack of me in these posts ver...,"[finding, lack, post, boring, position, often,...",1,0,1,0
2,INTP,'Good one _____ https://www.youtube.com/wat...,"[one, course, say, know, blessing, absolutely,...",0,0,1,0
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...","[intp, enjoyed, conversation, day, esoteric, g...",0,0,1,1
4,ENTJ,'You're fired.|||That's another silly misconce...,"[another, silly, misconception, approaching, l...",1,0,1,1
...,...,...,...,...,...,...,...
8670,ISFP,'https://www.youtube.com/watch?v=t8edHB_h908||...,"[always, think, cat, fi, doms, reason, website...",0,1,0,0
8671,ENFP,'So...if this thread already exists someplace ...,"[thread, already, exists, someplace, else, hec...",1,0,0,0
8672,INTP,'So many questions when i do these things. I ...,"[many, question, thing, would, take, purple, p...",0,0,1,0
8673,INFP,'I am very conflicted right now when it comes ...,"[conflicted, right, come, wanting, child, hone...",0,0,0,0


### google word2vec

In [ ]:
#Google News Word2Vec Model
import gensim.downloader as api
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten, Embedding, Input



# Load the Google News Word2Vec model
word2vec_model = api.load("word2vec-google-news-300")


In [16]:
def get_google_word2vec_matrices(data, model, max_length=800):
    """Convert sentences to a matrix of word vectors."""
    vector_size = model.vector_size
    embeddings = np.zeros((len(data), max_length, vector_size))
    
    for i, sentence in enumerate(data):
        words = sentence[:max_length]
        for j, word in enumerate(words):
            if word in model:
                embeddings[i, j, :] = model[word]
    
    return embeddings

### google word2vec 4 output

In [17]:
# # Prepare input data
X = get_google_word2vec_matrices(df['processed_posts'], word2vec_model)
Y = df[['I/E', 'N/S', 'F/T', 'P/J']]

In [14]:
Y.head()

,I/E,N/S,F/T,P/J
0,0,0,0,1
1,1,0,1,0
2,0,0,1,0
3,0,0,1,1
4,1,0,1,1


In [18]:
X.shape, Y.shape

((8675, 800, 300), (8675, 4))

In [19]:
X

array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.16308594,  0.02709961,  0.14453125, ...,  0.31445312,
          0.07861328, -0.03564453],
        [-0.0625    ,  0.04125977,  0.06030273, ..., -0.04882812,
         -0.10009766,  0.02246094],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[-0.01611328,  0.03173828, -0.21386719, ...,  0.1171875 ,
          0.13574219,  0.11425781],
        [ 0.08154297, -0.03149414, -0.2578125 , ..., -0.03295898,
         -0.04248047,  0.01098633],
        [-0.02148438, -0.00379944, -0.11474609, ..., -0.29882812,
         -0.13574219,  0.03613281],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  

In [20]:
# Split data into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)
# Create a single label that represents all combinations
# Y['combined'] = Y.apply(lambda row: ''.join(row.values.astype(str)), axis=1)
# X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y['combined'])

In [22]:
np.save('../data/word2vec/train/X_train.npy', X_train)
np.save('../data/word2vec/train/Y_train.npy', Y_train)
np.save('../data/word2vec/test/X_val.npy', X_val)
np.save('../data/word2vec/test/Y_val.npy', Y_val)

# np.save('../data/word2vec/train/X_train_balance.npy', X_train)
# np.save('../data/word2vec/train/Y_train_balance.npy', Y_train)
# np.save('../data/word2vec/test/X_val_balance.npy', X_val)
# np.save('../data/word2vec/test/Y_val_balance.npy', Y_val)

In [2]:
X_train = np.load('../data/word2vec/train/X_train.npy')
Y_train = np.load('../data/word2vec/train/Y_train.npy')
X_val = np.load('../data/word2vec/test/X_val.npy')
Y_val = np.load('../data/word2vec/test/Y_val.npy')

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, GlobalMaxPooling1D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

2024-05-04 16:51:58.435181: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [26]:
# Define the model architecture
input_shape = (X_train.shape[1], X_train.shape[2])  # max_length and vector_size
inputs = Input(shape=input_shape)

shared_layers = Sequential([
    Conv1D(filters=64, kernel_size=7, activation='relu'),
    BatchNormalization(),
    # Conv1D(filters=32, kernel_size=7, activation='relu'),
    # BatchNormalization(),
    # Conv1D(filters=16, kernel_size=7, activation='relu'),
    # BatchNormalization(),
    Flatten(),
    Dense(1024, activation='relu')
])

shared_output = shared_layers(inputs)

output1_layers = Sequential([
    Dense(128, activation='relu'),
    Dropout(0.2), 
    # Dense(8, activation='relu'),
    # Dropout(0.2), 
    Dense(1, activation='sigmoid'),
], name='output1')

output2_layers = Sequential([
    Dense(128, activation='relu'),
    Dropout(0.2), 
    # Dense(8, activation='relu'),
    # Dropout(0.2), 
    Dense(1, activation='sigmoid'),
], name='output2')

output3_layers = Sequential([
    Dense(128, activation='relu'),
    Dropout(0.5), 
    # Dense(8, activation='relu'),
    # Dropout(0.4), 
    Dense(1, activation='sigmoid'),
], name='output3')

output4_layers = Sequential([
    Dense(128, activation='relu'),
    Dropout(0.5), 
    # Dense(8, activation='relu'),
    # Dropout(0.3),  
    Dense(1, activation='sigmoid'),
], name='output4')

# Connect each sequential output to the shared output
output1 = output1_layers(shared_output)
output2 = output2_layers(shared_output)
output3 = output3_layers(shared_output)
output4 = output4_layers(shared_output)

optimizer = Adam(learning_rate=0.001)
# Assemble and compile the model
cnn_4_out_model = Model(inputs=inputs, outputs=[output1, output2, output3, output4], name='cnn_mbti')
cnn_4_out_model.compile(optimizer=optimizer,
              loss={'output1': 'binary_crossentropy', 'output2': 'binary_crossentropy',
                    'output3': 'binary_crossentropy', 'output4': 'binary_crossentropy'},
              metrics={'output1': 'accuracy', 'output2': 'accuracy', 'output3': 'accuracy', 'output4': 'accuracy'})

cnn_4_out_model.summary()

# Early stopping to prevent overfitting
#early_stopping_monitor = EarlyStopping(monitor='output1_accuracy', patience=2, verbose=2, mode='min', restore_best_weights=True)

Model: "cnn_mbti"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 800, 300)]   0           []                               
                                                                                                  
 sequential_2 (Sequential)      (None, 1024)         52171328    ['input_3[0][0]']                
                                                                                                  
 output1 (Sequential)           (None, 1)            131329      ['sequential_2[0][0]']           
                                                                                                  
 output2 (Sequential)           (None, 1)            131329      ['sequential_2[0][0]']           
                                                                                           

In [27]:
X_train.shape, Y_train.shape

((6940, 800, 300), (6940, 4))

In [28]:
y1_train, y2_train, y3_train, y4_train = Y_train[:, 0], Y_train[:,1], Y_train[:,2], Y_train[:,3]

In [29]:
# Train the model
batch_size = 64
epochs = 3
train_metrics = cnn_4_out_model.fit(x=X_train, y={'output1': y1_train, 'output2': y2_train, 
                        'output3': y3_train, 'output4': y4_train}, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    verbose=2
                    )

results = cnn_4_out_model.evaluate(x=X_val, 
                        y={'output1': Y_val[:,0], 'output2': Y_val[:,1], 
                        'output3': Y_val[:,2], 'output4': Y_val[:,3]}, 
                        verbose=2)

Epoch 1/3
109/109 - 73s - loss: 3.1076 - output1_loss: 0.7296 - output2_loss: 0.5671 - output3_loss: 0.9327 - output4_loss: 0.8781 - output1_accuracy: 0.7490 - output2_accuracy: 0.8500 - output3_accuracy: 0.5690 - output4_accuracy: 0.5733 - 73s/epoch - 670ms/step
Epoch 2/3
109/109 - 70s - loss: 1.5399 - output1_loss: 0.3457 - output2_loss: 0.2327 - output3_loss: 0.4556 - output4_loss: 0.5059 - output1_accuracy: 0.8354 - output2_accuracy: 0.8987 - output3_accuracy: 0.7888 - output4_accuracy: 0.7585 - 70s/epoch - 641ms/step
Epoch 3/3
109/109 - 74s - loss: 0.4713 - output1_loss: 0.1066 - output2_loss: 0.0548 - output3_loss: 0.1357 - output4_loss: 0.1743 - output1_accuracy: 0.9611 - output2_accuracy: 0.9814 - output3_accuracy: 0.9491 - output4_accuracy: 0.9347 - 74s/epoch - 677ms/step
55/55 - 8s - loss: 2.3732 - output1_loss: 0.5751 - output2_loss: 0.4469 - output3_loss: 0.6401 - output4_loss: 0.7110 - output1_accuracy: 0.7343 - output2_accuracy: 0.8548 - output3_accuracy: 0.6161 - output4

In [30]:
# Extract accuracy from each output
acc_output1 = train_metrics.history['output1_accuracy']
acc_output2 = train_metrics.history['output2_accuracy']
acc_output3 = train_metrics.history['output3_accuracy']
acc_output4 = train_metrics.history['output4_accuracy']

# Calculate the average accuracy over all outputs per epoch
average_accuracy_per_epoch = [np.mean(values) for values in zip(acc_output1, acc_output2, acc_output3, acc_output4)]

In [31]:
loss_total = results[0]
loss_output1, acc_output1 = results[1], results[5]
loss_output2, acc_output2 = results[2], results[6]
loss_output3, acc_output3 = results[3], results[7]
loss_output4, acc_output4 = results[4], results[8]
average_accuracy = (acc_output1 + acc_output2 + acc_output3 + acc_output4) / 4
print((acc_output1, acc_output2, acc_output3, acc_output4))
print(f'Average Accuracy: {average_accuracy}')

(0.7342939376831055, 0.8547550439834595, 0.6161383390426636, 0.5250720381736755)
Average Accuracy: 0.682564839720726


In [22]:
y1_predict, y2_predict, y3_predict, y4_predict = cnn_4_out_model.predict(X_val)

55/55 [==============================] - 7s 127ms/step


In [23]:
y1_predict.shape, y2_predict.shape, y3_predict.shape, y4_predict.shape

((1735, 1), (1735, 1), (1735, 1), (1735, 1))

In [24]:
def calculate_accuracy(y_true, y_pred):
    # Check if all four predicted classes match the true classes
    correct_predictions = np.all(y_true == y_pred, axis=1)
    accuracy = np.mean(correct_predictions)
    return accuracy

def threshold_prediction(value, threshold):
    if value >= threshold:
        return 1
    else:
        return 0

threshold = 0.5
y1_predict_thresholded = np.array([threshold_prediction(value, threshold) for value in y1_predict])
y2_predict_thresholded = np.array([threshold_prediction(value, threshold) for value in y2_predict])
y3_predict_thresholded = np.array([threshold_prediction(value, threshold) for value in y3_predict])
y4_predict_thresholded = np.array([threshold_prediction(value, threshold) for value in y4_predict])

y1_test_reshaped, y2_test_reshaped, y3_test_reshaped, y4_test_reshaped = Y_val[:,0].reshape(-1, 1), Y_val[:,1].reshape(-1, 1), Y_val[:,2].reshape(-1, 1), Y_val[:,3].reshape(-1, 1)

y_pred_combined = np.vstack((y1_predict_thresholded,
                                 y2_predict_thresholded,
                                 y3_predict_thresholded,
                                 y4_predict_thresholded)).T

y_test_combined = np.vstack((y1_test_reshaped.T,
                                y2_test_reshaped.T,
                                y3_test_reshaped.T,
                                y4_test_reshaped.T)).T

accuracy = calculate_accuracy(y_test_combined, y_pred_combined)

In [25]:
print(f'Accuracy across all four categories: {accuracy}')

Accuracy across all four categories: 0.21210374639769453
